In [ ]:
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
dir_in='f:/data/tc_wakes/database/info/'
dir_out='f:/data/tc_wakes/database/sst/'
#################################################################################
import datetime as dt
import xarray as xr
#from datetime import datetime
import pandas
import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
#import matplotlib.pyplot as plt
import numpy as np
import math
#from math import cos, radians
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
####################you will need to change some paths here!#####################
#list of input files
dir_in='f:/data/tc_wakes/database/info/'
dir_out='f:/data/tc_wakes/database/sat_data/'
dir_mur = 'F:/data/sst/jpl_mur/v4.1/'
#output files
#filename_out_nc='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/data_so_far.nc'
#filename_out_kml='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/data_so_far.kml'
#################################################################################
import datetime as dt
import xarray as xr
from datetime import datetime
import pandas
import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import math
#from math import cos, radians
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.mlab as mlab
palette = copy(plt.cm.jet)
palette.set_over('r', 1.0)
palette.set_under('g', 1.0)
palette.set_bad(alpha = 0.0)

In [10]:
dir_ccmp='F:/data/sat_data/ccmp/v02.0/Y'
date_1858 = dt.datetime(1858,11,17,0,0,0) # start date is 11/17/1958
dx=0.25
dy=0.25
dx_offset = -179.875
dy_offset = 78.3750
for root, dirs, files in os.walk(dir_in, topdown=False):
    #print(files)
    #for ii in range(40,90): 
    #print(root[31:35])
    for name in files:
        istart_storm=0
        #name = files[ii]
        fname_in=os.path.join(root, name)
        print(name,fname_in)
        dsx = xr.open_dataset(fname_in)
        lats = dsx.lat[0,:]
        lons = dsx.lon[0,:]
        dysince = dsx.time
        icycle=0
        minlon=min(lons.values)-7
        maxlon=max(lons.values)+7
        minlat=min(lats.values)-7
        maxlat=max(lats.values)+7
        if minlon<10. and maxlon>350.:  #wrapping around meridion need to cal new min/max lon
            minlon=max(lons[lons<180].values)+7
            maxlon=min(lons[lons>180].values)-7
            icycle=1 #set flag for wraparound
        dims=lats.shape
        print(dims)
        tdim=dims[0]
        tem_date=[0]*tdim #print(dysince.values)
        for i in range(0,tdim):
            tem_date[i]=date_1858+dt.timedelta(days=float(dysince[0,i].values))  #create new time array that can be queried for year etc
        minjdy = min(tem_date).timetuple().tm_yday  #create new time array that can be queried for year etc
        minyear =min(tem_date).year #create new time array that can be queried for year etc
        maxjdy = max(tem_date).timetuple().tm_yday  #create new time array that can be queried for year etc
        maxyear =max(tem_date).year  #create new time array that can be queried for year etc
        dif = max(tem_date)-min(tem_date)
        tdim=int(dif.days)  
        print(minjdy,maxjdy,minlon,maxlon,minlat,maxlat)
        ds_new=[]
        new_coord=[]
        for incr_dy in range(-5,tdim+25): #from -5 to 25 days after
            storm_date = tem_date[0]+dt.timedelta(days=incr_dy)            
            #print(storm_date)
            syr=str(storm_date.year)
            smon=str(storm_date.month)
            sdym=str(storm_date.day)
            sjdy=str(storm_date.timetuple().tm_yday)
            fname_tem=syr + smon.zfill(2) + sdym.zfill(2) + '090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
            mur_filename = dir_mur + syr + '/' + sjdy.zfill(3) + '/' + fname_tem
            fname_tem='/CCMP_Wind_Analysis_' + syr + smon.zfill(2) + sdym.zfill(2) + '_V02.0_L3.0_RSS.nc'
            ccmp_filename = dir_ccmp + syr + '/M' + smon.zfill(2) + fname_tem      
            print(ccmp_filename)
            ds=xr.open_dataset(ccmp_filename,drop_variables='nobs')  #don't need nobs
            ds_day=ds.mean(dim='time')  #average over day
            #select lat/lon region
            if icycle==0:
                ds2=ds_day.sel(latitude=slice(minlat,maxlat),longitude=slice(minlon,maxlon))
            else:
                ds2=ds_day.sel(latitude=slice(minlat,maxlat),longitude=(ds_day.longitude < minlon) | (ds_day.longitude > maxlon))
            #print(ds2)
            ds_new.append(ds2)
            new_coord.append(storm_date)
        combined = xr.concat(ds_new, dim='time')
        combined.coords['time'] = new_coord
        fname_out=dir_out + 'ccmp' + root[31:35] + name[0:3] + '.nc'
        print(fname_out)
        combined.to_netcdf(fname_out)
        #break
    #break
#print(ds_new)


001annual_storm_info.nc f:/data/tc_wakes/database/info/2004\001annual_storm_info.nc
(26,)
271 277 56.299998585134745 74.3999984934926 -13.999999843537807 2.8000000938773155
F:/data/sat_data/ccmp/v02.0/Y2003/M09/CCMP_Wind_Analysis_20030923_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M09/CCMP_Wind_Analysis_20030924_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M09/CCMP_Wind_Analysis_20030925_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M09/CCMP_Wind_Analysis_20030926_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M09/CCMP_Wind_Analysis_20030927_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M09/CCMP_Wind_Analysis_20030928_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M09/CCMP_Wind_Analysis_20030929_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M09/CCMP_Wind_Analysis_20030930_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M10/CCMP_Wind_Analysis_20031001_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M10/CCMP_Wind_Analysis_20031002_V02.0

F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031210_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031211_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031212_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031213_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031214_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031215_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031216_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031217_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031218_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031219_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031220_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2003/M12/CCMP_Wind_Analysis_20031221_V02.0_L3.0

F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040101_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040102_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040103_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040104_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040105_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040106_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040107_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040108_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040109_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040110_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040111_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040112_V02.0_L3.0

F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040128_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040129_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040130_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040131_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M02/CCMP_Wind_Analysis_20040201_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M02/CCMP_Wind_Analysis_20040202_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M02/CCMP_Wind_Analysis_20040203_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M02/CCMP_Wind_Analysis_20040204_V02.0_L3.0_RSS.nc
f:/data/tc_wakes/database/sat_data/ccmp2004007.nc
008annual_storm_info.nc f:/data/tc_wakes/database/info/2004\008annual_storm_info.nc
(79,)
24 44 35.499999050050974 64.89999870583415 -43.3999991863966 -8.29999965801835
F:/data/sat_data/ccmp/v02.0/Y2004/M01/CCMP_Wind_Analysis_20040119_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02

F:/data/sat_data/ccmp/v02.0/Y2004/M02/CCMP_Wind_Analysis_20040226_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M02/CCMP_Wind_Analysis_20040227_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M02/CCMP_Wind_Analysis_20040228_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M02/CCMP_Wind_Analysis_20040229_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M03/CCMP_Wind_Analysis_20040301_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M03/CCMP_Wind_Analysis_20040302_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M03/CCMP_Wind_Analysis_20040303_V02.0_L3.0_RSS.nc
f:/data/tc_wakes/database/sat_data/ccmp2004009.nc
011annual_storm_info.nc f:/data/tc_wakes/database/info/2004\011annual_storm_info.nc
(20,)
41 46 119.29999717697501 153.69999672099948 -28.199999526143074 -6.799999691545963
F:/data/sat_data/ccmp/v02.0/Y2004/M02/CCMP_Wind_Analysis_20040205_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2004/M02/CCMP_Wind_Analysis_20040206_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp

...

F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161104_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161105_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161106_V02.0_L3.0_RSS.nc
f:/data/tc_wakes/database/sat_data/ccmp2016076.nc
077annual_storm_info.nc f:/data/tc_wakes/database/info/2016\077annual_storm_info.nc
(63,)
278 293 286.1000014953315 330.5000008158386 16.19999948143959 65.99999868124723
F:/data/sat_data/ccmp/v02.0/Y2016/M09/CCMP_Wind_Analysis_20160929_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M09/CCMP_Wind_Analysis_20160930_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161001_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161002_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161003_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161004_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v0

F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161016_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161017_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161018_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161019_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161020_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161021_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161022_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161023_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161024_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161025_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161026_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161027_V02.0_L3.0

F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161109_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161110_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161111_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161112_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161113_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161114_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161115_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161116_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161117_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161118_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161119_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161120_V02.0_L3.0

F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161028_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161029_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161030_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M10/CCMP_Wind_Analysis_20161031_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161101_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161102_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161103_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161104_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161105_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161106_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161107_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M11/CCMP_Wind_Analysis_20161108_V02.0_L3.0

F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161209_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161210_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161211_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161212_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161213_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161214_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161215_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161216_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161217_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161218_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2016/M12/CCMP_Wind_Analysis_20161219_V02.0_L3.0_RSS.nc
f:/data/tc_wakes/database/sat_data/ccmp2016087.nc
090annual_storm_info.nc f: